##📌 LIMPIEZA TRIPADVISOR

> Add blockquote



Este notebook contiene código para unificar y procesar reseñas de viajes de Tripadvisor obtenidas de varios archivos CSV. Realiza las siguientes tareas:

1.  **Carga de datos:** Lee archivos CSV con nombres que siguen el patrón `trip_reviews_final_*.csv`.
2.  **Limpieza de texto:** Elimina emojis y normaliza mayúsculas en los comentarios.
3.  **Clasificación de experiencia:** Intenta categorizar cada reseña en un tipo de experiencia (Hotel, Playa, Tour, etc.) basado en palabras clave en el título y el comentario.
4.  **Unificación:** Combina todas las reseñas procesadas en un único DataFrame.
5.  **Guardar resultados:** Guarda el DataFrame unificado en un nuevo archivo CSV.
6.  **Análisis básico:** Realiza un conteo de reseñas por ciudad y tipo de experiencia.

##📌 LIBRERIAS

In [ ]:
!pip install emoji dateparser langdetect

In [ ]:
import os, glob, re
import pandas as pd
from functools import lru_cache
from datetime import datetime
import os
import re
import glob
import pandas as pd
from functools import lru_cache

##📌 DESARROLLO

## ⚙️ Procesamiento y Unificación de Datos

Esta celda contiene las funciones y la lógica principal para cargar, limpiar, normalizar, clasificar y unificar las reseñas de Tripadvisor de varios archivos CSV, generando un archivo de salida consolidado.

In [ ]:

# ==========================
# Configuración
# ==========================
DATA_DIR = os.getcwd()
FILE_PATTERN = os.path.join(DATA_DIR, "trip_reviews_final*.csv")
OUTPUT_FILE = os.path.join(DATA_DIR, "tripadvisor_reviews_unificado.csv")

files = glob.glob(FILE_PATTERN)
print("Archivos encontrados:", [os.path.basename(f) for f in files])

# ==========================
# Utilidades
# ==========================

def limpiar_emoji(texto: str) -> str:
    EMOJI_RE = re.compile(
        "["
        "\U0001F300-\U0001F5FF"
        "\U0001F600-\U0001F64F"
        "\U0001F680-\U0001F6FF"
        "\U0001F700-\U0001F77F"
        "\U0001F780-\U0001F7FF"
        "\U0001F900-\U0001F9FF"
        "\U00002600-\U000026FF"
        "\U00002700-\U000027BF"
        "\U00002B00-\U00002BFF"
        "]+"
    )
    if not isinstance(texto, str):
        return ""
    return re.sub(r"\s+", " ", EMOJI_RE.sub("", texto)).strip()

def normalizar_mayusculas(texto: str) -> str:
    """Convierte el texto a minúsculas y capitaliza el inicio y tras ., ?, !, ;"""
    if not isinstance(texto, str):
        return ""
    texto = texto.strip().lower()
    def _capitalize(match):
        return match.group(0).upper()
    # Capitaliza la primera letra
    texto = re.sub(r'(^[a-záéíóúüñ])', _capitalize, texto)
    # Capitaliza después de . ! ? ;
    texto = re.sub(
        r'([\.!?;]\s*)([a-záéíóúüñ])',
        lambda m: m.group(1) + m.group(2).upper(),
        texto
    )
    return texto

# Localizador de columnas
def get_col(df, candidates):
    cols = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand.lower() in cols:
            return cols[cand.lower()]
    for c in df.columns:
        for cand in candidates:
            if cand.lower() in c.lower():
                return c
    return None

# Normalizar ciudad desde nombre de archivo
def _city_from_filename(fname: str) -> str:
    m = re.match(r"trip_reviews_final_(.+?)\.csv$", os.path.basename(fname), re.IGNORECASE)
    city = (m.group(1) if m else "").strip().lower()
    mapping = {
        "canarias": "Gran Canaria",
        "malaga": "Málaga",
        "barcelona": "Barcelona",
        "madrid": "Madrid",
        "mallorca": "Mallorca",
        "sevilla": "Sevilla",
        "tenerife": "Tenerife",
        "valencia": "Valencia",
    }
    return mapping.get(city, city.capitalize())

# Experiencia
EXPERIENCE_KEYWORDS = {
    "Hotel": ["hotel","hostal","albergue","inn","auberge","hôtel","pousada","pensione","otel","hostel"],
    "Playa": ["playa","beach","spiaggia","praia","strand","plage","chiringuito"],
    "Tour": ["tour","excursion","excursión","ruta","visita guiada","guided tour","excursao","gira","itinerario","paseo","guia"],
    "Deporte": ["deporte","deportes","sport","sports","surf","tenis","fútbol","football","soccer","gym","gimnasio","baloncesto","pádel","paddle","ciclismo","hiking","senderismo", "kayak","snorkel"],
    "Cultura": ["cultura","cultural","culture","teatro","ópera","opera","arquitectura","historia","patrimonio","heritage","monumento","exposición","exposicion"],
    "Comida": ["comida","food","gastronomía","gastronomia","restaurante","restaurant","cena","almuerzo","desayuno","tapas","platos","dishes","cuisine","eat","manger","mangiare"],
    "Museo": ["museo","museum","musée","museu","musei","galería","galeria","gallery"],
    "Vida nocturna": ["discoteca","nightlife","clubbing","fiesta","pub","bares","copas","afterwork","club","night club"],
    "Ocio": ["ocio","entretenimiento","diversión","leisure","entertainment","evento","eventos","planes","show", "taller","activity","tripulacion","espectaculo","zoologico","acuario","fotografía"],
}
EXPERIENCE_ORDER = ["Hotel","Playa","Tour","Deporte","Cultura","Comida","Museo","Vida nocturna","Ocio"]

EXP_PATTERNS = {cat: r"(?:" + r"|".join(map(re.escape, kws)) + r")" for cat, kws in EXPERIENCE_KEYWORDS.items()}

def clasificar_experiencia_vectorizado(serie_texto: pd.Series) -> pd.Series:
    flags = {}
    for cat in EXPERIENCE_ORDER:
        flags[cat] = serie_texto.str.contains(EXP_PATTERNS[cat], case=False, na=False, regex=True)
    flags_df = pd.DataFrame(flags, index=serie_texto.index)
    result = pd.Series("Desconocido", index=serie_texto.index)
    mask_restante = pd.Series(True, index=serie_texto.index)
    for cat in EXPERIENCE_ORDER:
        m = mask_restante & flags_df[cat]
        result[m] = cat
        mask_restante &= ~flags_df[cat]
        if not mask_restante.any():
            break
    return result

# ==========================
# Procesar un archivo
# ==========================
def process_file(filepath: str) -> pd.DataFrame:
    city = _city_from_filename(filepath)
    df = pd.read_csv(filepath, encoding="utf-8", on_bad_lines="skip")

    col_coment = get_col(df, ["comentario"])
    col_titulo = get_col(df, ["titulo"])
    col_fecha  = get_col(df, ["fecha"])

    if not col_coment:
        raise ValueError(f"No se encontró la columna de comentarios en {os.path.basename(filepath)}")

    # Limpiar y normalizar texto
    texto_crudo  = df[col_coment].astype(str)
    texto_limpio = texto_crudo.map(limpiar_emoji).map(normalizar_mayusculas)

    # Experiencia
    if col_titulo:
        combinado = (df[col_titulo].astype(str).fillna("") + " " + texto_crudo).str.strip()
    else:
        combinado = texto_crudo
    experiencia = clasificar_experiencia_vectorizado(combinado)

    # Fecha (sin parser avanzado, solo limpia nulos)
    fecha = df[col_fecha] if col_fecha else pd.Series(pd.NA, index=df.index)

    # Construir salida
    out = pd.DataFrame({
        "Titulo": df[col_titulo] if col_titulo else "",
        "Comentario": texto_limpio,
        "Fecha": fecha,
        "Ciudad": city,
        "Experiencia": experiencia,
        "Fuente": "Tripadvisor"
    })

    return out

# ==========================
# Ejecutar todo
# ==========================
processed, errores = [], {}
for fp in files:
    try:
        df_out = process_file(fp)
        processed.append(df_out)
        print("OK:", os.path.basename(fp), "->", len(df_out), "filas")
    except Exception as e:
        errores[os.path.basename(fp)] = str(e)
        print("ERROR:", os.path.basename(fp), "->", e)

if processed:
    final_df = pd.concat(processed, ignore_index=True)
    final_df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")
    print("\n✅ Archivo final guardado en:", OUTPUT_FILE, "| Filas:", len(final_df))
else:
    print("❌ No se generó salida.")

if errores:
    print("\nArchivos con error:")
    for k, v in errores.items():
        print(" -", k, ":", v)

# Vista rápida
try:
    display(final_df.head())
except:
    pass


Archivos encontrados: ['trip_reviews_final_canarias.csv', 'trip_reviews_final_malaga.csv', 'trip_reviews_final_barcelona.csv', 'trip_reviews_final_madrid.csv', 'trip_reviews_final_valencia.csv', 'trip_reviews_final_tenerife.csv', 'trip_reviews_final_mallorca.csv', 'trip_reviews_final_sevilla.csv']
OK: trip_reviews_final_canarias.csv -> 9778 filas
OK: trip_reviews_final_malaga.csv -> 11999 filas
OK: trip_reviews_final_barcelona.csv -> 49413 filas
OK: trip_reviews_final_madrid.csv -> 44541 filas
OK: trip_reviews_final_valencia.csv -> 4668 filas
OK: trip_reviews_final_tenerife.csv -> 14178 filas
OK: trip_reviews_final_mallorca.csv -> 2643 filas
OK: trip_reviews_final_sevilla.csv -> 20239 filas

✅ Archivo final guardado en: /Users/thaliacamargo/tripadvisor_reviews_unificado.csv | Filas: 157459


,Titulo,Comentario,Fecha,Ciudad,Experiencia,Fuente
0,Paseos En Camello Por Las Dunas De Maspalomas,Una experiencia muy buena y sobre todo al rese...,2025-04-23,Gran Canaria,Tour,Tripadvisor
1,Paseos En Camello Por Las Dunas De Maspalomas,"Magnifica experiencia, personal muy amable y a...",2025-04-23,Gran Canaria,Tour,Tripadvisor
2,Paseos En Camello Por Las Dunas De Maspalomas,"El dia que teniamos reservado hizo malisimo, l...",2025-04-23,Gran Canaria,Tour,Tripadvisor
3,Paseos En Camello Por Las Dunas De Maspalomas,"Muy buen tour, una nueva experiencia. Excelent...",2025-04-23,Gran Canaria,Tour,Tripadvisor
4,Paseos En Camello Por Las Dunas De Maspalomas,Es lo peor que puedes hacer estas fomentando e...,2024-12-23,Gran Canaria,Tour,Tripadvisor


**bold text**## 📊 Análisis Básico por Ciudad y Experiencia

Esta celda calcula y muestra la cantidad de reseñas por ciudad y por tipo de experiencia, presentando los resultados en una tabla pivote para facilitar la visualización.

In [ ]:
# Conteo por ciudad y experiencia
conteo = final_df.groupby(["Ciudad","Experiencia"]).size().reset_index(name="Cantidad")

# Si prefieres ver como tabla pivote (más legible)
pivot = conteo.pivot(index="Ciudad", columns="Experiencia", values="Cantidad").fillna(0).astype(int)
print(pivot)


Experiencia   Comida  Cultura  Deporte  Desconocido  Hotel  Museo  Ocio  \
Ciudad                                                                    
Barcelona       1725     1038     1166         4164   4769    784   732   
Gran Canaria     167       17      298          844    423      0   134   
Madrid          1898      839       58         3049    680    295   332   
Mallorca         205       28       31          342    158      3    55   
Málaga           751      537       47         1393    461    406   154   
Sevilla          644      741      192         1866    549    485   739   
Tenerife         891       94      789         3465    861     75   999   
Valencia         282      135       23          455    179     51    71   

Experiencia   Playa   Tour  Vida nocturna  
Ciudad                                     
Barcelona      2945  31869            221  
Gran Canaria    769   7121              5  
Madrid           95  36780            515  
Mallorca        110   1706   